In [ ]:
import requests as r
%matplotlib inline
import pandas as pd
from nltk.corpus import stopwords

In [ ]:
#we don't need these. I left them here just in case.
# appID = '1844787218898682'
# appSecret = 'b3e7536d6dd029c773441d5271318dec'

In [ ]:
#this is where you have to go to get a token to test with. You need to go there to get a new access_token every time you run the script.
#https://developers.facebook.com/tools/explorer/145634995501895/?method=GET&path=me%2Ffeed&version=v2.10

In [ ]:
access_token = 'EAACEdEose0cBAGHWZAec37neJdulqMocs6LZCMQUrn8H0Er2nNtSFLBeFmtR4DCiZC9gVYSTXeKykqlbiOU9oUhEZAnl5vJ55aYOLYcOXOPbveW5sc9VdPh9C0TVecAOZBV0L7wdQbdZB9mjI9JtEQJEuIEtYrWycrLw5CpBMw2u29VKxyVQhLhZCsz5OvqfloZD'

In [ ]:
allData = []
firstURL = 'https://graph.facebook.com/v2.10/me/posts?access_token={}'.format(access_token)
response = r.get(firstURL)
print(response)
postData = response.json()
allData = allData + postData['data']
print(len(allData))

In [ ]:
while 'paging' in postData:
    nextURL = postData['paging']['next']
    response = r.get(nextURL)
    print(response)
    postData = response.json()
    allData = allData + postData['data']
    print(len(allData))

In [ ]:
df = pd.DataFrame(allData)

In [ ]:
df.created_time = pd.to_datetime(df.created_time)

In [ ]:
df = df[df.created_time > datetime.datetime(2011,6,1)].copy()

In [ ]:
text = df[df.message == df.message].message.str.cat(sep=' ')

In [ ]:
text2 = text.replace('\n\n',' ').replace('\n',' ').replace(u"\'","").lower()

In [ ]:
test = pd.Series(text2.split(' '))
urls = test[test.str.contains('http')|test.str.contains('www')].values
test = test[~(test.str.contains('http')|test.str.contains('www'))].copy()
text2 = test.str.cat(sep=' ')

In [ ]:
import string
for x in list(string.punctuation):
    text2 = text2.replace(x,' ')

In [ ]:
words_list = text2.split(' ')

In [ ]:
words_list = pd.Series(words_list)

In [ ]:
words_list = words_list[words_list != ''].copy()

In [ ]:
stop = stopwords.words('english')
final_words_list = words_list[~words_list.isin(stop)]

In [ ]:
#top words used in my Facebook posts
final_words_list.value_counts()[:20]

In [ ]:
from textblob import TextBlob
dfNoNan = df[df.message == df.message].copy()
dfNoNan['polarity'] = dfNoNan['message'].map(lambda x: TextBlob(x).sentiment.polarity)
dfNoNan['subjectivity'] = dfNoNan['message'].map(lambda x: TextBlob(x).sentiment.subjectivity)

In [ ]:
import math
result = {}
for x in range(math.ceil(len(dfNoNan.id) / 50)):
    newIDs = dfNoNan.id[50*x:50*(x+1)]
    stringIDs = ','.join(newIDs)
    firstURL = 'https://graph.facebook.com/v2.10?ids={}&fields=shares.limit(5000).summary(true),likes.limit(5000).summary(true),comments.limit(5000).summary(true)&access_token={}'.format(stringIDs,access_token)
    response = r.get(firstURL)
    print(response)
    postData = response.json()
    result.update(postData)
    print(len(result))

In [ ]:
def getLikesName(cell):
    total_likes_name = len(result[cell]['likes']['data'])
    return total_likes_name

def getLikesCount(cell):
    total_likes_count = result[cell]['likes']['summary']['total_count']
    return total_likes_count

def getCommentsName(cell):
    total_comments_name = len(result[cell]['comments']['data'])
    return total_comments_name

def getCommentsCount(cell):
    total_comments_count = result[cell]['comments']['summary']['total_count']
    return total_comments_count

In [ ]:
dfNoNan['likesName'] = dfNoNan.id.map(getLikesName)
dfNoNan['likesCount'] = dfNoNan.id.map(getLikesCount)
dfNoNan['commentsName'] = dfNoNan.id.map(getCommentsName)
dfNoNan['commentsCount'] = dfNoNan.id.map(getCommentsCount)

In [ ]:
#total number of posts on facebook (this counts shares and posts)
df['id'].count()

In [ ]:
#total number of times posts have been liked
dfNoNan.likesCount.sum()

In [ ]:
likes_people = []
comments_people = []
for postID in dfNoNan.id:
    likes_people = likes_people + [x['name'] for x in result[postID]['likes']['data']]
    comments_people = comments_people + [x['from']['name'] for x in result[postID]['comments']['data']]

In [ ]:
likes_people = pd.Series(likes_people)
comments_people = pd.Series(comments_people)

In [ ]:
#top people based on comments on my Facebook Posts
comments_people.value_counts()[:20]

In [ ]:
#top people based on likes on my Facebook Posts
likes_people.value_counts()[:20]

In [ ]:
#top tweets based on comments
dfNoNan.sort_values('commentsCount',ascending=False)[:10][['created_time','message','story','commentsCount']]

In [ ]:
#top tweets based on likes
dfNoNan.sort_values('likesCount',ascending=False)[:10][['created_time','message','story','likesCount']]

In [ ]:
import datetime
df.created_time = pd.to_datetime(df.created_time)

In [ ]:
timedf = df.copy()

In [ ]:
timedf.index = df.created_time

In [ ]:
test = timedf.groupby(timedf.index.year.astype(str) + timedf.index.week.astype(str)).count()

In [ ]:
def fixIndex(cell):
    if len(cell) == 5:
        return cell[:4] + '0' + cell[4]
    else:
        return cell
test.index = test.index.map(fixIndex)

In [ ]:
new_week_index = []
#eventually pull these dates from latest tweet, go back until earliest tweet
for x in range(2011,2018):
    for y in range(0,53):
        new_week_index.append(str(x) + str(y))

In [ ]:
new_week_index = pd.Series(new_week_index).map(fixIndex)

In [ ]:
test = test.reindex(new_week_index, fill_value=0)

In [ ]:
#facebook activity over time (grouped by week)
test['id'].plot()

In [ ]:
test2 = timedf.groupby(timedf.index.dayofweek).count()

In [ ]:
#Monday is 0. Sunday is 6.
#Facebook posts by day of week
test2['id'].plot()

In [ ]:
#this is because I am in a time zone that is -7 GMT
#this will need to be changed based on user time zone
test3 = timedf.groupby(timedf.index.hour - 7).count()

In [ ]:
#0 = midnight. 12 = noon.
#facebook posts by time of day
test3['id'].plot()

In [ ]:
test4 = timedf.groupby(timedf.index.month).count()

In [ ]:
#facebook posts by month
test4['id'].plot()

In [ ]:
allData = []
firstURL = 'https://graph.facebook.com/v2.10/10153451802413555/likes?access_token={}'.format(access_token)
response = r.get(firstURL)
print(response)
postData = response.json()
allData = allData + postData['data']
print(len(allData))

In [ ]:
while 'paging' in postData:
    nextURL = postData['paging']['next']
    response = r.get(nextURL)
    print(response)
    postData = response.json()
    allData = allData + postData['data']
    print(len(allData))

In [ ]:
pages = pd.DataFrame(allData)

In [ ]:
#get the feed
allData = []
firstURL = 'https://graph.facebook.com/v2.10/10153451802413555/feed?access_token={}'.format(access_token)
response = r.get(firstURL)
print(response)
postData = response.json()
allData = allData + postData['data']
print(len(allData))

In [ ]:
while 'paging' in postData:
    nextURL = postData['paging']['next']
    response = r.get(nextURL)
    print(response)
    postData = response.json()
    allData = allData + postData['data']
    print(len(allData))

In [ ]:
feed = pd.DataFrame(allData)

In [ ]:
import math
result = {}
for x in range(math.ceil(len(feed.id) / 50)):
    newIDs = feed.id[50*x:50*(x+1)]
    stringIDs = ','.join(newIDs)
    firstURL = 'https://graph.facebook.com/v2.10?ids={}&fields=shares.limit(5000).summary(true),likes.limit(5000).summary(true),comments.limit(5000).summary(true)&access_token={}'.format(stringIDs,access_token)
    response = r.get(firstURL)
    print(response)
    postData = response.json()
    result.update(postData)
    print(len(result))

In [ ]:
#this is your Facebook feed (not just your posts, but everything that shows up on your wall)
#this can go back into your other script and you can look at all of the same things as just the posts script
#result

In [ ]:
#get og.likes (only returns 3 likes, they are all associated with GoFundMe. I don't know why.)
allData = []
firstURL = 'https://graph.facebook.com/v2.10/10153451802413555/og.likes?since=1167609600&access_token={}'.format(access_token)
response = r.get(firstURL)
print(response)
postData = response.json()
allData = allData + postData['data']
print(len(allData))

In [ ]:
#scroll down in activity viewer to get all likes and comments
#use copy(document.body.innerHTML); to copy text
#paste it somewhere. Save it.
#now analyze it.

with open('C:/Users/bodil/Desktop/FacebookPostsLikes.html','r',encoding='utf-8') as file:
    data = file.read()

In [ ]:
from bs4 import BeautifulSoup as BS
soup = BS(data,"lxml")

In [ ]:
soup2 = soup.find_all('div',class_='fbTimelineLogBody')

In [ ]:
profiles = soup2[0].find_all('a',class_="profileLink")

In [ ]:
people = soup2[0].find_all('td',class_='_5ep5')
posts = soup2[0].find_all('div',class_='fsm')

In [ ]:
people2 = []
for x in people:
    people2.append(x.text)

def parse_comments(cell):
    flag = False
    if 'his own comment' in cell:
        return 'his own comment'
    if 'comment' in cell and 'commented' not in cell:
        return 'comment'
    elif 'post' in cell:
        return 'post'
    elif 'life event' in cell:
        return 'life event'
    elif 'link' in cell:
        return 'link'
    elif 'album' in cell:
        return 'album'
    elif 'photo' in cell:
        return 'photo'
    elif 'live video' in cell:
        return 'live video'
    elif 'video' in cell:
        return 'video'
    elif 'campaign' in cell:
        return 'campaign'
    elif 'status' in cell:
        return 'status'
    elif 'bio' in cell:
        return 'bio'
    elif 'timeline' in cell:
        return 'timeline'
    elif 'friend request' in cell:
        return 'friend request'
    elif 'profile picture' in cell:
        return 'profile picture'
    elif 'an event' in cell:
        return 'an event'
    elif 'a list' in cell:
        return 'a list'
    elif 'an entry' in cell:
        return 'an entry'
    elif 'a note' in cell:
        return 'a note'
    elif 'Quotations' in cell:
        return 'quotations'
    elif 'phone number' in cell:
        return 'phone number'
    elif 'Website' in cell:
        return 'website'
    elif 'memory' in cell:
        return 'memory'
    elif 'poll' in cell:
        return 'poll'
    elif 'hometown' in cell:
        return 'hometown'
    elif 'Page' in cell:
        return 'page'
    else:
        return None

people2 = pd.DataFrame(people2)
# people2[~(people2[0].map(parse_comments)==people2[0].map(parse_comments))]
people2['media'] = people2[0].map(parse_comments)

In [ ]:
#top media that you interact with
people2.media.value_counts()[:10]

In [ ]:
people2.rename(columns={0:'event_text'},inplace=True)

In [ ]:
def parse_action(cell):
    if 'liked' in cell:
        return 'liked'
    elif 'likes' in cell:
        return 'likes'
    elif 'reacted to' in cell:
        return 'reacted to'
    elif 'is going to' in cell:
        return 'is going to'
    elif 'was mentioned in a' in cell:
        return 'was mentioned in a'
    elif 'commented on' in cell:
        return 'commented on'
    elif 'posted in' in cell:
        return 'posted in'
    elif 'became friends with' in cell:
        return 'became friends with'
    elif 'replied to' in cell:
        return 'replied to'
    elif 'added' in cell:
        return 'added'
    elif 'shared a post to your' in cell:
        return 'shared a post to your'
    elif 'was tagged in' in cell:
        return 'was tagged in'
    elif 'shared' in cell:
        return 'shared'
    elif 'wrote on your' in cell:
        return 'wrote on your'
    elif 'wrote on' in cell:
        return 'wrote on'
    elif 'updated' in cell:
         return 'updated'
    elif 'sent' in cell:
        return 'sent'
    elif 'changed' in cell:
        return 'changed'
    elif 'posted a video to' in cell:
        return 'posted a video to'
    elif 'posted something via' in cell:
        return 'posted something via'
    elif 'voted for' in cell:
        return 'voted for'
    elif 'was feeling' in cell:
        return 'was feeling'
    elif 'reviewed' in cell:
        return 'reviewed'
    elif 'created' in cell:
        return 'created'
    elif 'was tagged at' in cell:
        return 'was tagged at'
    elif 'voted on' in cell:
        return 'voted on'
    elif 'was with' in cell:
        return 'was with'
    elif 'is now using Facebook in' in cell:
        return 'is now using Facebook in'
    elif 'is interested in' in cell:
        return 'is interested in'
    elif 'was' in cell:
        return 'was'
# people2[~(people2.event_text.map(parse_action)==people2.event_text.map(parse_action))]
people2['event'] = people2.event_text.map(parse_action)

In [ ]:
#top interactions on Facebook
people2.event.value_counts()[:10]

In [ ]:
people2['date'] = people2.event_text.map(lambda x: x.split('.')[-1])

In [ ]:
for x in people2.index:
    if people2.loc[x]['event_text'] != people2.loc[x]['event_text']:
        people2.loc[x,'name'] = None
    else:
        test = people2.loc[x]['event_text']
        if people2.loc[x,'event'] == 'became friends with':
            person = test[test.find(people2.loc[x]['event']) + len(people2.loc[x]['event']) + 1:test.find('.')]
            people2.loc[x,'name'] = person
        elif ' own ' in people2.loc[x,'event_text'] or ' his ' in people2.loc[x,'event_text'] or ' her ' in people2.loc[x,'event_text']:
            people2.loc[x,'name'] = None
        elif people2.loc[x,'event'] == 'added':
            people2.loc[x,'name'] = None
        elif people2.loc[x,'media'] == None:
            people2.loc[x,'name'] = None
        else:
            person = test[test.find(people2.loc[x]['event']) + len(people2.loc[x]['event']) + 1:test.find(people2.loc[x]['media']) - 3]
            people2.loc[x,'name'] = person

In [ ]:
#people you interact with the most
people2[people2.name != ''].name.value_counts()[:20]

In [ ]:
#things you interact with the most
people2.media.value_counts()[:10]

In [ ]:
posts2 = []
for x in posts:
    posts2.append(x.text)

In [ ]:
text = ' '.join(posts2)
text2 = text.replace('\n\n',' ').replace('\n',' ').replace(u"\'","").lower()
test = pd.Series(text2.split(' '))
urls = test[test.str.contains('http')|test.str.contains('www')].values
test = test[~(test.str.contains('http')|test.str.contains('www')|test.str.contains('.com'))].copy()
text2 = test.str.cat(sep=' ')
import string
for x in list(string.punctuation):
    text2 = text2.replace(x,' ')
    words_list = text2.split(' ')
words_list = pd.Series(words_list)
words_list = words_list[words_list != ''].copy()
stop = stopwords.words('english')
final_words_list = words_list[~words_list.isin(stop)]

In [ ]:
#words in Facebook interactions you have had
final_words_list.value_counts()

In [ ]:
#look at likes and comments over time for a single post (top liked and commented posts--to get a sense for timeline of liking and posting)
#when do I like, comment, or post things? Any patterns?
#look at URLs I have posted about (top URLs...?)
#look at hashtags I have used (how am I treating #aslkdflksd--I need to pull them out)